In [ ]:
import numpy as np
import pandas as pd
import statistics as st
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm

import warnings
import math

from tabulate import tabulate
from typing import Union

from Utils import *
from FinancialMetrics import *

## Settings

In [ ]:
warnings.filterwarnings('ignore')

## Constants

In [ ]:
ROLLING_WINDOW_SIZE = 180
CLOSING_PRICE_CSV_NAME = 'NASDAQ-100-CLOSING-PRICES.csv' # The file must be in the same directory of the notebook

## Data Pre-Processing
The indexes dataframe contains all the value of each index in the range of Dates between 04/01/2017 and 24/05/2022

### Get the Closing Price Dataset

In [ ]:
closing_prices = pd.read_csv(CLOSING_PRICE_CSV_NAME)
closing_prices.head()

### Calculate the log returns

In [ ]:
nasdaq100_returns = pd.read_csv('indexes.csv')
#nasdaq100_returns = get_log_returns(CLOSING_PRICE_CSV_NAME)
nasdaq100_returns.fillna(np.nan, inplace=True)
nasdaq100_returns.head()
nasdaq100_returns.head().style.applymap(color_negative_red)

## Rolling Linear Regression
Creiamo un dataframe (rollingDataFrame) contenente le prime ROLLING_WINDOW_SIZE righe del dataframe indexes e creiamo una funzione che, a seconda dei parametri, costruisce portfoli basati su alpha, beta, r^2 o errore.
All'interno della lista indexNames inseriamo gli indici presenti in questi primi ROLLING_WINDOW_SIZE giorni.

    • rollingDataFrame: contiene le prime ROLLING_WINDOW_SIZE righe del dataframe indexes;
    • indexNames: contiene gli indici sempre presenti all'interno del Nasdaq nei primi ROLLING_WINDOW_SIZE giorni
    • selector: str in ['alpha', 'beta', 'r2', 'error']

In [ ]:
def get_window_returns(days):
    window_returns = nasdaq100_returns.iloc[days: days + ROLLING_WINDOW_SIZE]
    window_returns = window_returns.reset_index().drop(['index'], axis=1)

    # Remove titles that are not in the Nasdaq-100 window range
    window_returns.dropna(axis=1, how='any', inplace=True)

    # Get the name of the columns
    titles = window_returns.columns.tolist()

    # Remove the first two element in indexNames (Dates, NDX Index) because I don't need them
    # mantaining NDX Index in tmp
    titles = titles[2:]

    return window_returns, titles

## Example: First 180 days

In [ ]:
rolling_df, titles = get_window_returns(0)
print("In the first {} days, {} stocks will be taken from the index and analyzed.".format(ROLLING_WINDOW_SIZE, len(rolling_df.columns)))
rolling_df.head().style.applymap(color_negative_red)

## Portfolio Implementation

In [ ]:
def portfolio_ranked(selector, selector_columns, days):
    """
    Build a portfolio, based on the selector, by taking the titles included in the index in the range [days; days + ROLLING_WINDOW_SIZE]
    :param selector: The selector to use in order to build the rank and select the titles
    :param selector_columns: Optional values to use in the specific selector
    :param days: The number of days to skip 
    """

    rolling_df, titles = get_window_returns(days)

    # Build the dataframe
    rank_df = pd.DataFrame(columns=['Title', selector] + selector_columns)
    ndx_returns = rolling_df.iloc[:, 1].values

    for title in titles:
        title_returns = rolling_df.iloc[0 : ROLLING_WINDOW_SIZE, rolling_df.columns.get_loc(title)]

        ndx_returns = sm.add_constant(ndx_returns)
        model = sm.OLS(title_returns, ndx_returns)
        result = model.fit()

        if selector == 'max_r2':
            rank_df = rank_df.append({'Title': title, selector: result.rsquared}, ignore_index=True)
        elif selector == 'min_r2_and_high_specific_risk':
            rank_df = rank_df.append({'Title': title, selector: result.rsquared, 'specificRisk': result.resid.std() ** 2}, ignore_index=True)
        elif selector == 'max_specific_risk':
            rank_df = rank_df.append({'Title': title, selector: result.resid.std() ** 2}, ignore_index=True)
        elif selector == 'max_beta' or selector == 'min_beta':
            rank_df = rank_df.append({'Title': title, selector: result.params[1] ** 2 * ndx_returns.std() ** 2}, ignore_index=True)
        elif selector == 'high_systematic_risk' or selector == 'low_systematic_risk':
            rank_df = rank_df.append({'Title': title, selector: result.params[1] ** 2 * ndx_returns.std() ** 2}, ignore_index=True)
        elif selector == 'positive_alpha':
            rank_df = rank_df.append({'Title': title, selector: result.params[0]}, ignore_index=True)
        elif selector == 'positive_and_significant_alpha':
            rank_df = rank_df.append({'Title': title, selector: result.params[0], 'alpha_significance':  result.pvalues[0]}, ignore_index=True)
        elif selector == 'positive_alpha_and_high_beta':
            rank_df = rank_df.append({'Title': title, selector: result.params[1], 'alpha': result.params[0],'alpha_significance':  result.pvalues[0]}, ignore_index=True)
        elif selector == 'absolute_returns':
            #totalReturn = np.sum(title_returns)
            #rank_df = rank_df.append({'Title': title, selector: totalReturn}, ignore_index=True)
            rank_df = rank_df.append({'Title': title, selector: get_absolute_return(title_returns)}, ignore_index=True)
        
    # Do the rank
    if selector == 'max_r2' or selector == 'max_specific_risk' or selector == 'absolute_returns' or selector == 'max_beta' or selector == 'positive_alpha':
        winners = rank_df.sort_values(by=selector, ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
        selected_values = winners[selector].tolist()
    elif selector == 'min_beta':
        winners = rank_df.sort_values(by=selector, ascending=True).head(10)
        selected_titles = winners['Title'].tolist()
        selected_values = winners[selector].tolist()
    elif selector == 'high_systematic_risk' or selector == 'low_systematic_risk':
        ascending = False
        if 'low' in selector:
            ascending = True
        winners = rank_df.sort_values(by=selector, ascending=ascending).head(10)
        selected_titles = winners['Title'].tolist()
        selected_values = winners[selector].tolist()
    elif selector == 'positive_and_significant_alpha':
        rank_df = rank_df[rank_df['alpha_significance'] < 0.05]
        rank_df = rank_df[rank_df[selector] > 0]
        if(rank_df.shape[1] < 10):
            raise Exception("Not enought titles")
        winners = rank_df.sort_values(by=selector, ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
        selected_values = winners[selector].tolist()
    elif selector == 'positive_alpha_and_high_beta':
        rank_df = rank_df[rank_df['alpha'] > 0]
        winners = rank_df.sort_values(by=selector, ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
        selected_values = winners[selector].tolist()
    elif selector == 'min_r2_and_high_specific_risk':
        rank_df = rank_df.sort_values(by=[selector], ascending=True)
        rank_df = rank_df.head(int(len(titles) * 1/3))
        winners = rank_df.sort_values(by='specificRisk', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
        selected_values = list(zip(winners[selector].tolist(), zip(winners['specificRisk'].tolist())))
        
    return selected_titles, selected_values
    

In [ ]:
selectors_optional_variables = {
    'min_r2_and_high_specific_risk' : ['specific_risk'],
    'positive_and_significant_alpha' : ['alpha_significance'],
    'positive_alpha_and_high_beta' : ['alpha', 'alpha_significance']
}

In [ ]:
def portfolio_builder(selector):
    portfolio_returns = []
    portfolio = pd.DataFrame(columns=['Dates'] + [f'Title{i}' for i in range(1,11)] + ['Returns'])

    selector_columns = []
    
    if selector in selectors_optional_variables.keys():
        selector_columns = selectors_optional_variables[selector]

    titles, _ = portfolio_ranked(selector, selector_columns, 0)
    
    days_range = nasdaq100_returns.shape[0] - ROLLING_WINDOW_SIZE
    for days in range(7, days_range, 7):
        nasdaq100_window_returns = nasdaq100_returns[titles].iloc[ROLLING_WINDOW_SIZE + days - 7 : ROLLING_WINDOW_SIZE + days]
        for _, row in nasdaq100_window_returns.iterrows():
            if np.isnan(row.sum()):
                print("NaN in row")
                print(row)
                print("\n")
            portfolio_returns.append(row.sum() / 10)

        portfolio_row = {'Dates': nasdaq100_returns.iloc[ROLLING_WINDOW_SIZE + days]['Dates']}
        portfolio_row.update({f'Title{i}': titles[i - 1] for i in range(1,11)})
        # portfolio_row.update({'Returns': portfolio_returns[-1]}) # DECIDE WHAT TO DO
        portfolio = portfolio.append(portfolio_row, ignore_index=True)
        titles, _ = portfolio_ranked(selector, selector_columns, days)
    return portfolio, portfolio_returns
    

## Portfolio Selectors
- max_r2: desc..

In [ ]:
selectors = [
    'max_r2',
    'absolute_returns',
    'min_r2_and_high_specific_risk', 
    'max_specific_risk',
    'max_beta',
    'min_beta', 
    'positive_alpha',
    'high_systematic_risk',
    'low_systematic_risk',
    'positive_and_significant_alpha',
    'positive_alpha_and_high_beta'
]

In [ ]:
base_metrics = pd.DataFrame(columns=['Portfolio Title', 'Annualized Returns', 'Annualized Volatility'])
advanced_metrics = pd.DataFrame(columns=['Portfolio Title', 'Sharpe Ratio', 'Efficiency', 'MDD', 'Var 90', 'Var 95', 'Var 99', 'IR'])

In [ ]:
portfolios_analysis = {}
ndx_returns = nasdaq100_returns[181:]['NDX Index'].tolist()

In [ ]:
for selector in selectors:
    try:
        print(f'Buildindg {selector} portfolio')
        portfolio, returns = portfolio_builder(selector)
        portfolios_analysis[selector] = returns
        basic_row = get_base_metrics(selector, returns)
        base_metrics = base_metrics.append(basic_row, ignore_index=True)

        advanced_row = get_advanced_metrics(selector, returns, ndx_returns)
        advanced_metrics = advanced_metrics.append(advanced_row, ignore_index=True)
    except Exception as e:
        print(f'Cannot build the portfolio for the selector {selector}: {str(e)}')


In [ ]:
print(tabulate(base_metrics, headers='keys', tablefmt='psql')) 

In [ ]:
print(tabulate(advanced_metrics, headers='keys', tablefmt='psql')) 

In [ ]:
def portfolios_comparison(money, portfolios):
    dates = np.array(nasdaq100_returns[181:]['Dates']) 
    ndx_returns = np.array(nasdaq100_returns[181:]['NDX Index'])
    
    returns = []
    dates_to_show = [dates[i] for i in np.linspace(0, len(dates) - 1, 20).astype(int)]

    returns.append(money * math.exp(ndx_returns[0]))
    for i in range(1, len(dates)):
        returns.append(returns[i-1] * math.exp(ndx_returns[i]))

    plt.figure(figsize=(20,10))
    plt.plot(dates, returns, label='NDX Index')

    for title, portfolio_returns in portfolios.items():
        returns = []
        returns.append(money * math.exp(portfolio_returns[0]))
        for i in range(1, len(dates)):
            returns.append(returns[i-1] * math.exp(portfolio_returns[i]))
        plt.plot(dates, returns, label=title)

    plt.xlabel("Date")
    plt.xticks(dates_to_show, rotation=45)
    plt.ylabel("Value ($)")
    plt.legend()
    plt.title("Portfolios Comparison")
    # Aggiungere legenda log return

In [ ]:
portfolios_comparison(100, portfolios_analysis)

In [ ]:
# Mostrare in una tabella anche il max, il min e ultimo valore disponibile
# Riportare lo sharp ration negativo di questo periodo
# per quanto riguarda il selector absolute_return mostrare risultati come nell'immagine nella cartella Financial
    # Oppure magari provare: al posto di considerare tutti i 180 giorni provare solo con gli ultimi 10